# 1. Data Extraction

In [ ]:
import os
import requests
import pandas as pd
import time
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")
BASE_URL = 'https://www.alphavantage.co/query'

In [ ]:
# Fetch data from Alpha Vantage (general use)
def fetch_data(params, is_csv=False):
    params['apikey'] = API_KEY
    response = requests.get(BASE_URL, params=params)
    if is_csv:
        return response.text
    try:
        return response.json()
    except ValueError:
        return {}

def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def save_json(data, base_dir, filename, key=None):
    if key:
        data = data.get(key, {})
    if not data:
        print(f"❌ No data for {filename}")
        return
    df = pd.DataFrame.from_dict(data, orient='index')
    df.index.name = 'date'
    full_path = os.path.join(base_dir, filename)
    df.to_csv(full_path)
    print(f"✅ Saved: {full_path}")

def save_flat_json(data, base_dir, filename):
    if not data:
        print(f"❌ No data for {filename}")
        return
    df = pd.DataFrame([data])
    full_path = os.path.join(base_dir, filename)
    df.to_csv(full_path, index=False)
    print(f"✅ Saved: {full_path}")

def save_csv(content, base_dir, filename):
    full_path = os.path.join(base_dir, filename)
    with open(full_path, "w", encoding="utf-8") as f:
        f.write(content)
    print(f"✅ Saved: {full_path}")

def fetch_stock_data(symbol):
    base_dir = f"data/{symbol}"
    ensure_dir(base_dir)

    fundamentals_dir = os.path.join(base_dir, "fundamentals")
    prices_dir = os.path.join(base_dir, "prices")
    quotes_dir = os.path.join(base_dir, "quotes")
    news_dir = os.path.join(base_dir, "news")

    for folder in [fundamentals_dir, prices_dir, quotes_dir, news_dir]:
        ensure_dir(folder)

    endpoints = [
        ("TIME_SERIES_DAILY", "prices", "daily.csv", "Time Series (Daily)"),
        ("TIME_SERIES_DAILY_ADJUSTED", "prices", "daily_adjusted.csv", "Time Series (Daily)"),
        ("TIME_SERIES_WEEKLY", "prices", "weekly.csv", "Weekly Time Series"),
        ("TIME_SERIES_WEEKLY_ADJUSTED", "prices", "weekly_adjusted.csv", "Weekly Adjusted Time Series"),
        ("TIME_SERIES_MONTHLY", "prices", "monthly.csv", "Monthly Time Series"),
        ("TIME_SERIES_MONTHLY_ADJUSTED", "prices", "monthly_adjusted.csv", "Monthly Adjusted Time Series"),
        ("GLOBAL_QUOTE", "quotes", "quote.csv", None),
        ("OVERVIEW", "fundamentals", "overview.csv", None),
        ("INCOME_STATEMENT", "fundamentals", "income_statement.csv", None),
        ("BALANCE_SHEET", "fundamentals", "balance_sheet.csv", None),
        ("CASH_FLOW", "fundamentals", "cash_flow.csv", None),
        ("EARNINGS", "fundamentals", "earnings.csv", None),
        ("DIVIDENDS", "fundamentals", "dividends.csv", None),
        ("SPLITS", "fundamentals", "splits.csv", None),
        ("INSIDER_TRANSACTIONS", "fundamentals", "insider_transactions.csv", None)
    ]

    for func, folder, filename, key in endpoints:
        params = {'function': func, 'symbol': symbol}

        # Get full history for time series
        if "TIME_SERIES" in func and "INTRADAY" not in func:
            params["outputsize"] = "full"

        data = fetch_data(params)
        full_dir = os.path.join(base_dir, folder)

        if isinstance(data, dict):
            if key:
                save_json(data, full_dir, f"{symbol}_{filename}", key)
            else:
                if 'annualReports' in data:
                    pd.DataFrame(data['annualReports']).to_csv(os.path.join(full_dir, f"{symbol}_{filename}"), index=False)
                elif 'quarterlyEarnings' in data:
                    pd.DataFrame(data['quarterlyEarnings']).to_csv(os.path.join(full_dir, f"{symbol}_{filename}"), index=False)
                else:
                    save_flat_json(data, full_dir, f"{symbol}_{filename}")
        time.sleep(15)

def fetch_news_sentiment(symbol):
    news_dir = f"data/{symbol}/news"
    ensure_dir(news_dir)
    params = {'function': 'NEWS_SENTIMENT', 'tickers': symbol, 'limit': 1000}
    data = fetch_data(params)
    if data and "feed" in data:
        df = pd.DataFrame(data["feed"])
        df.to_csv(os.path.join(news_dir, f"{symbol}_news_sentiment.csv"), index=False)
        print(f"✅ Saved: {symbol}_news_sentiment.csv")
    time.sleep(15)

def fetch_earnings_transcript(symbol, quarter="2024Q1"):
    fundamentals_dir = f"data/{symbol}/fundamentals"
    ensure_dir(fundamentals_dir)
    params = {'function': 'EARNINGS_CALL_TRANSCRIPT', 'symbol': symbol, 'quarter': quarter}
    data = fetch_data(params)
    save_flat_json(data, fundamentals_dir, f"{symbol}_earnings_transcript.csv")
    time.sleep(15)

def fetch_macro_data():
    base_dir = "data/macros"
    ensure_dir(base_dir)
    indicators = [
        "REAL_GDP", "REAL_GDP_PER_CAPITA", "TREASURY_YIELD", "FEDERAL_FUNDS_RATE", "CPI", "INFLATION",
        "RETAIL_SALES", "DURABLES", "UNEMPLOYMENT", "NONFARM_PAYROLL"
    ]
    for ind in indicators:
        params = {'function': ind}
        data = fetch_data(params)
        save_json(data, base_dir, f"{ind.lower()}.csv")
        time.sleep(15)

def fetch_calendars():
    base_dir = "data/macros"
    ensure_dir(base_dir)
    endpoints = [
        ("EARNINGS_CALENDAR", "earnings_calendar.csv", {"horizon": "3month"}),
        ("IPO_CALENDAR", "ipo_calendar.csv", {})
    ]
    for func, filename, params in endpoints:
        params.update({'function': func})
        content = fetch_data(params, is_csv=True)
        save_csv(content, base_dir, filename)
        time.sleep(15)

In [ ]:
def main():
    symbols = ["AAPL", "AMZN", "GOOG", "MSFT", "META", "NVDA", "TSLA", "SPY", "URTH"]
    for symbol in symbols:
        try:
            print(f"\n📈 Fetching data for {symbol}...")
            fetch_stock_data(symbol)
            fetch_news_sentiment(symbol)
            fetch_earnings_transcript(symbol)
        except Exception as e:
            print(f"❌ Error with {symbol}: {e}")
            continue

    print("\n🌍 Fetching macroeconomic indicators...")
    fetch_macro_data()

    print("\n📆 Fetching IPO and Earnings calendars...")
    fetch_calendars()

    print("\n🎉 All data fetched successfully.")

if __name__ == '__main__':
    main()

## Market Overview 

In [ ]:
import os
import requests
import pandas as pd
import time
from dotenv import load_dotenv

# Cargar API Key desde .env
load_dotenv()
API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")
BASE_URL = 'https://www.alphavantage.co/query'

# Lista de símbolos
symbols = [ "AAPL", "AMZN", "GOOGL", "MSFT", "META", "NVDA", "TSLA"]

# Función para hacer request a Alpha Vantage
def fetch_data(params):
    params['apikey'] = API_KEY
    response = requests.get(BASE_URL, params=params)
    try:
        return response.json()
    except Exception as e:
        print(f"❌ Error parsing JSON: {e}")
        return {}

# Guardar el JSON plano en formato vertical clave-valor (una fila por clave)
def save_flat_json_vertical(data, base_dir, filename):
    if not data or "Note" in data or "Error Message" in data:
        print(f"❌ Error en la respuesta para {filename}")
        return False
    df = pd.DataFrame(data.items(), columns=["Key", "Value"])
    os.makedirs(base_dir, exist_ok=True)
    path = os.path.join(base_dir, filename)
    df.to_csv(path, index=False)
    print(f"✅ Guardado: {path}")
    return True

# Proceso principal
def main():
    for symbol in symbols:
        print(f"\n📥 Descargando OVERVIEW para {symbol}...")
        success = False
        attempts = 0

        while not success and attempts < 3:
            params = {'function': 'OVERVIEW', 'symbol': symbol}
            data = fetch_data(params)
            success = save_flat_json_vertical(data, f"data/{symbol}/fundamentals", f"{symbol}_overview.csv")
            if not success:
                print(f"🔁 Esperando para reintentar {symbol}...")
                time.sleep(20)
            attempts += 1

        time.sleep(15)  # Tiempo de espera entre símbolos para no superar el límite gratuito

if __name__ == "__main__":
    main()


In [ ]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import time

# Cargar API Key
load_dotenv()
API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")
BASE_URL = 'https://www.alphavantage.co/query'

# Función general de descarga
def fetch_data(params):
    params['apikey'] = API_KEY
    response = requests.get(BASE_URL, params=params)
    try:
        return response.json()
    except ValueError:
        return {}

# Asegurar carpeta
def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Función aislada para GOOGL news
def fetch_news_sentiment(symbol="GOOGL"):
    news_dir = f"data/{symbol}/news"
    ensure_dir(news_dir)

    print(f"📥 Descargando news sentiment para {symbol}...")
    params = {
        'function': 'NEWS_SENTIMENT',
        'tickers': symbol,
        'limit': 1000
    }

    data = fetch_data(params)

    if data and "feed" in data and len(data["feed"]) > 0:
        df = pd.DataFrame(data["feed"])
        df.to_csv(os.path.join(news_dir, f"{symbol}_news_sentiment.csv"), index=False)
        print(f"✅ Guardado correctamente: {symbol}_news_sentiment.csv")
    else:
        print(f"❌ No se encontraron datos válidos para {symbol}")

    time.sleep(15)

# Ejecutar solo esta descarga
fetch_news_sentiment("GOOGL")


📥 Descargando news sentiment para GOOGL...
❌ No se encontraron datos válidos para GOOGL


### **News Extraction** - 2 Week-Window-Frame (03-2022 to 05-2025)

In [111]:
#!/usr/bin/env python3
import os, csv, time, requests, argparse
import pandas as pd
from datetime import datetime, timedelta

BASE_URL = "https://www.alphavantage.co/query"
API_KEY  = os.getenv("ALPHA_VANTAGE_API_KEY", "TU_ALPHA_KEY")

def iso_param(dt: datetime) -> str:
    return dt.strftime("%Y%m%dT%H%M")

def fetch_news(ticker, time_from, time_to, limit=1000):
    params = {
        "function":  "NEWS_SENTIMENT",
        "tickers":   ticker,
        "apikey":    API_KEY,
        "time_from": time_from,
        "time_to":   time_to,
        "limit":     limit
    }
    r = requests.get(BASE_URL, params=params)
    r.raise_for_status()
    return r.json().get("feed", [])

def download_all(ticker, start_date, end_date, window_weeks, out_csv):
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt   = datetime.strptime(end_date,   "%Y-%m-%d")
    window   = timedelta(weeks=window_weeks)
    all_articles = []
    current = start_dt

    while current < end_dt:
        nxt = min(current + window, end_dt)
        tf = iso_param(current); tt = iso_param(nxt)
        print(f"Fetching {ticker} from {tf} to {tt}…")
        batch = fetch_news(ticker, tf, tt)
        if batch:
            all_articles.extend(batch)
            print(f"  → {len(batch)} articles")
        else:
            print("  → no articles")
        current = nxt
        time.sleep(12)

    df = pd.DataFrame(all_articles)
    df.drop_duplicates(subset=["url","time_published"], inplace=True)
    df.to_csv(out_csv, sep=';', index=False, quoting=csv.QUOTE_ALL, encoding='utf-8')
    print(f"Saved {len(df)} unique articles to '{out_csv}'")




In [ ]:
if __name__ == "__main__":
    # Parámetros
    ticker     = "META"
    start_date = "2022-03-01"
    end_date   = datetime.now().strftime("%Y-%m-%d")
    window     = 1   # semanas
    output_csv = f"{ticker}_news_{start_date}_to_{end_date}.csv"

    # Llamada a la descarga
    download_all(ticker, start_date, end_date, window, output_csv)
    print(f"✅ Descarga completada para {ticker}, guardado en '{output_csv}'")


## 1.1. Data Enrichment

In [ ]:
import requests
import pandas as pd
import os

# ────── CONFIG ───────────────────────────────────────────────
API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")  # or replace with your key: "YOUR_KEY_HERE"
SYMBOL = "AAPL"  # Change to your desired ticker
df_to_enrich = df_expl.copy()  # <── SELECT YOUR INPUT SENTIMENT DF HERE

# ────── STEP 1: DOWNLOAD FUNDAMENTALS ────────────────────────
urls = {
    "income": f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={SYMBOL}&apikey={API_KEY}",
    "balance": f"https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={SYMBOL}&apikey={API_KEY}",
    "cashflow": f"https://www.alphavantage.co/query?function=CASH_FLOW&symbol={SYMBOL}&apikey={API_KEY}"
}

financial_reports = {}

def fetch_quarterly_report(report_type):
    print(f"⏳ Downloading {report_type} report...")
    response = requests.get(urls[report_type])
    data = response.json()
    if "quarterlyReports" in data:
        df = pd.DataFrame(data["quarterlyReports"])
        financial_reports[report_type] = df
        print(f"✅ {report_type}: {df.shape[0]} quarters")
    else:
        print(f"❌ Failed to load {report_type} report:", data)

for report in ["income", "balance", "cashflow"]:
    fetch_quarterly_report(report)

# ────── STEP 2: CLEANING & RATIO CALCULATION ────────────────
def first_available_column(df, candidates):
    for col in candidates:
        if col in df.columns:
            return col
    raise ValueError(f"None of {candidates} found in DataFrame.")

def get_quarter_label(date):
    month = date.month
    quarter = (month - 1) // 3 + 1
    return f"{date.year}Q{quarter}"

def force_numeric(df):
    for c in df.columns:
        if c != "fiscalDateEnding":
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

df_income = financial_reports["income"].copy()
df_balance = financial_reports["balance"].copy()
df_cashflow = financial_reports["cashflow"].copy()

for df in [df_income, df_balance, df_cashflow]:
    df["fiscalDateEnding"] = pd.to_datetime(df["fiscalDateEnding"])
    df = force_numeric(df)

df_income = df_income[df_income["fiscalDateEnding"] >= "2022-01-01"]
df_balance = df_balance[df_balance["fiscalDateEnding"] >= "2022-01-01"]
df_cashflow = df_cashflow[df_cashflow["fiscalDateEnding"] >= "2022-01-01"]

df = df_balance.merge(df_income, on="fiscalDateEnding").merge(df_cashflow, on="fiscalDateEnding")

# Define fallback-safe columns
net_income_col = first_available_column(df, ["netIncome", "netIncomeFromContinuingOperations", "profitLoss"])
shares_col = first_available_column(df, ["commonStockSharesOutstanding"])

# Financial ratios
df["current_ratio"] = df["totalCurrentAssets"] / df["totalCurrentLiabilities"]
df["quick_ratio"] = (df["cashAndShortTermInvestments"] + df["currentNetReceivables"]) / df["totalCurrentLiabilities"]
df["debt_to_assets"] = df["totalLiabilities"] / df["totalAssets"]
df["debt_to_equity"] = df["totalLiabilities"] / df["totalShareholderEquity"]
df["gross_margin"] = df["grossProfit"] / df["totalRevenue"]
df["operating_margin"] = df["operatingIncome"] / df["totalRevenue"]
df["net_profit_margin"] = df[net_income_col] / df["totalRevenue"]
df["return_on_assets"] = df[net_income_col] / df["totalAssets"]
df["return_on_equity"] = df[net_income_col] / df["totalShareholderEquity"]
df["operating_cash_flow_ratio"] = df["operatingCashflow"] / df["totalCurrentLiabilities"]
df["free_cash_flow"] = df["operatingCashflow"] - df["capitalExpenditures"]
df["book_value_per_share"] = df["totalShareholderEquity"] / df[shares_col]
df["revenue_per_share"] = df["totalRevenue"] / df[shares_col]
df["eps"] = df[net_income_col] / df[shares_col]
df["quarter"] = df["fiscalDateEnding"].apply(get_quarter_label)

df_ratios = df[[
    "quarter", "eps", "book_value_per_share", "revenue_per_share",
    "current_ratio", "quick_ratio", "debt_to_assets", "debt_to_equity",
    "gross_margin", "operating_margin", "net_profit_margin",
    "return_on_assets", "return_on_equity", "operating_cash_flow_ratio", "free_cash_flow"
]]

# ────── STEP 3: LOAD WEEKLY PRICES ───────────────────────────
price_path = f"{BASE_PATH}/data/extract/stocks/{SYMBOL}/prices/{SYMBOL}_weekly_adjusted.csv"
df_price = pd.read_csv(price_path, usecols=["date", "5. adjusted close"])
df_price["date"] = pd.to_datetime(df_price["date"])
df_price = df_price[df_price["date"] >= "2022-01-01"].sort_values("date")
df_price = df_price.rename(columns={"5. adjusted close": "adjusted_close"})
df_price["quarter"] = df_price["date"].apply(get_quarter_label)

# ────── STEP 4: VALUATION RATIOS & ENRICHMENT ───────────────
df_final = df_price.merge(df_ratios, on="quarter", how="left")
df_final["PER"] = df_final["adjusted_close"] / df_final["eps"]
df_final["PB_ratio"] = df_final["adjusted_close"] / df_final["book_value_per_share"]
df_final["PS_ratio"] = df_final["adjusted_close"] / df_final["revenue_per_share"]
df_final["week"] = df_final["date"].dt.to_period("W").apply(lambda r: r.start_time)

# ────── STEP 5: MERGE WITH NEWS SENTIMENT DF ────────────────
df_to_enrich["time_published"] = pd.to_datetime(df_to_enrich["time_published"])
df_to_enrich["week"] = df_to_enrich["time_published"].dt.to_period("W").apply(lambda r: r.start_time)
df_merged = pd.merge(df_to_enrich, df_final.drop(columns=["date"]), on="week", how="left")

# ────── STEP 6: EXPORT AND PREVIEW ──────────────────────────
output_file = f"{SYMBOL}_enriched_sentiment_with_fundamentals.csv"
df_merged.to_csv(output_file, index=False)
print(f"✅ Final enriched DataFrame saved as {output_file}")
df_merged.head(5)
